# Obteniendo el TOTAL de personas totalmente vacunadas (2 dosis) y de muertes confirmadas por COVID-19 de cada uno de los 24 departamentos del Perú

En este notebook se busca obtener un dataset con el total de **fallecidos** y **totalmente vacunados** **(2 dosis)** por cada **departamento** de todo el Perú. De la misma manera se agrega el **número de habitantes** de cada departamento, para calcular la **tasa de mortalidad por 100 000 habitantes** y el **porcentaje de vacunación** respectivo.

Se considera como **completamente vacunado** a todas las personas que recibieron 2 dosis de vacunación.

## 0. Cargar librerías

In [1]:
import pandas as pd
import dask.dataframe as dd

## 1. Cargar direcciones de RawData

In [2]:
vac_url = 'RawData/TB_VACUNACION_COVID19.csv'
fal_url = 'RawData/fallecidos_covid.csv'

## 2. Procesar del dataset de fallecidos

### 2.1. Leer el dataset

In [3]:
# Leer solamente la columna fecha de departamento
fal_col = ['DEPARTAMENTO']
df_fal =  pd.read_csv(fal_url, sep = ';', usecols = fal_col, dtype = {'DEPARTAMENTO': 'category'})
del fal_col

# Agregamos un contador para cada fallecido
df_fal['fallecidos'] = 1

df_fal.head()

,DEPARTAMENTO,fallecidos
0,LIMA,1
1,CALLAO,1
2,LIMA,1
3,JUNIN,1
4,LIMA,1


### 2.2. Procesar el dataset

Se cuenta a cada caso con el contador creado anteriormente respecto a los 24 departamentos.

In [4]:
df_fal = df_fal.groupby('DEPARTAMENTO').count()
df_fal

,fallecidos
DEPARTAMENTO,
AMAZONAS,1280
ANCASH,6751
APURIMAC,1526
AREQUIPA,9747
AYACUCHO,2157
CAJAMARCA,4175
CALLAO,10091
CUSCO,4829
HUANCAVELICA,1183


Ahora tenemos el TOTAL de fallecidos por cada departamento del Perú.

## 3. Procesar el dataset de vacunados

No existe una manera directa de calcular el total de personas completamente vacunadas por departamento. Para lograrlo se prevee lo siguiente:

- El dataset de vacunados (RawData/TB_VACUNACION_COVID19.csv) solo proporciona información sobre el centro de vacunación llamado **'id_centro_vacunacion'**. NO el departamento.

- El dataset de los centros de vacunación [(RawData/TB_CENTRO_VACUNACION.csv)]() se puede utilizar para hacer **"match"** del **'id_centro_vacunacion'** del dataset de vacunados con la variable **'id_ubigeo'**. Que es una variable numérica de 0 a 1894 que representa a cada distrito.

- Finalmente, con el dataset de UBIGEO [(RawData/TB_UBIGEOS.csv)]() es posible reemplazar a cada **'id_ubigeo'** con el departamento respectivo.

### 3.1. Leer el dataset


In [5]:
# Leemos solo la columna dosis del dataset de vacunados usando dask
vac_col = ['id_eess', 'dosis']
df_vac = dd.read_csv(vac_url, sep = ",", usecols = vac_col, 
                     dtype = {'id_eess':'int32','dosis': 'int8'})

# Déjamos solamente a personas con 3 o menos dosis
df_vac = df_vac[df_vac['dosis'] <= 3]

del vac_col
df_vac.head()

,id_eess,dosis
0,21367,2
1,15019,1
2,4298,1
3,16954,2
4,4331,1


In [6]:
df = pd.crosstab(index= df_vac['id_eess'], columns=df_vac['dosis'])
df

col_0,1,2,3
row_0,,,
6,2019,2003,0
13,187,128,1
15,282,199,1
28,1441,2953,4295
35,9,7,0
...,...,...,...
28541,59,49,0
28551,138,93,0
28553,35,35,30


### 3.2. Procesar los datasets de **UBIGEO** y **CENTROS DE VACUNACIÓN**.

#### 3.2.1. Cargar direcciones de RawData

In [7]:
ubigeo_url = 'RawData/TB_UBIGEOS.csv'
vaccenter_url = 'RawData/TB_EESS.csv'

#### 3.2.2. Procesar los datasets

Para el dataset de UBIGEO solo se necesita las columnas 'id_ubigeo' y 'departamento'. Para el dataset de CENTROS DE VACUNACIÓN solo se necesitan las columnas 'id_centro de vacunación' e 'id_ubigeo'.

In [8]:
# Leemos los datasets de ubigeo y centros de vacunación
ubigeo = pd.read_csv(ubigeo_url, usecols = ['id_ubigeo', 'departamento'])
vaccenter = pd.read_csv(vaccenter_url, usecols= ['id_eess','id_ubigeo'])

# Unimos ambos dataset mediante 'id_ubigeo'
vaccenter = vaccenter.merge(ubigeo, on = 'id_ubigeo', how = 'left')
del ubigeo, vaccenter['id_ubigeo']

vaccenter.head()

,id_eess,departamento
0,17241,LORETO
1,14145,LORETO
2,7083,LORETO
3,549,LORETO
4,3207,LORETO


Ahora con el nuevo dataset **(vaccenter)** es posible encontrar el departamento correspondiente a 'id_centro de vacunacion' del dataset de vacunados **(df_vac)**.

#### 3.2.3. Encontrar el departamento correspondiente de todas las personas vacunadas

In [9]:
df_vac = df_vac.merge(vaccenter, on = 'id_eess', how = 'left')
del df_vac['id_eess']
del vaccenter

df_vac.head()

,dosis,departamento
0,2,LIMA
1,1,LIMA
2,1,CALLAO
3,2,CUSCO
4,1,LIMA


## 4. Agrupar a todos los vacunados por departamento

### 4.1. Realizar una crosstab de número de dosis por departamento

In [10]:
df_vac = pd.crosstab(index=[df_vac['departamento']], columns=[df_vac['dosis']])
df_vac

col_0,1,2,3
row_0,,,
AMAZONAS,254079,214434,18043
ANCASH,907529,824080,84031
APURIMAC,335102,281123,25093
AREQUIPA,1140676,1018102,120436
AYACUCHO,405434,307663,15034
CAJAMARCA,1005794,826116,46189
CALLAO,1039222,945876,128395
CUSCO,952209,799366,61303
HUANCAVELICA,238716,182878,12699


## 5. Unir el dataset del TOTAL de muertes confirmadas y TOTAL vacunados por departamento

In [11]:
vacfal_dep = pd.concat([df_fal, df_vac], axis=1)

vacfal_dep.reset_index(level=0, inplace=True)
vacfal_dep.rename(columns = {'index':'departamentos', 1:'dosis_1', 2: 'dosis_2', 3:'dosis_3'}, inplace = True)

del df_fal, df_vac
vacfal_dep

,departamentos,fallecidos,dosis_1,dosis_2,dosis_3
0,AMAZONAS,1280,254079,214434,18043
1,ANCASH,6751,907529,824080,84031
2,APURIMAC,1526,335102,281123,25093
3,AREQUIPA,9747,1140676,1018102,120436
4,AYACUCHO,2157,405434,307663,15034
5,CAJAMARCA,4175,1005794,826116,46189
6,CALLAO,10091,1039222,945876,128395
7,CUSCO,4829,952209,799366,61303
8,HUANCAVELICA,1183,238716,182878,12699
9,HUANUCO,2715,499979,392333,31353


## 6. Procesar el dataset por el número de habitantes correspondiente a cada departamento

Datos de población estimada para el 2020 obtenida del reporte [«PERÚ:Estimaciones y Proyecciones de Población por Departamento, Provincia y Distrito, 2018 - 2020»](https://www.inei.gob.pe/media/MenuRecursivo/publicaciones_digitales/Est/Lib1715/Libro.pdf). Página 39 estimaciones de población total proyectada para 2020. Fuente: [INEI](https://www.inei.gob.pe/).

### 6.1. Agregar un diccionario con el número de habitantes de cada departamento

In [12]:
dic_dep = {
    "AMAZONAS"	:	426806,
    "ANCASH"	:	1180638,
    "APURIMAC"	:	430736,
    "AREQUIPA"	:	1497438,
    "AYACUCHO"	:	668213,
    "CAJAMARCA"	:	1453711,
    "CALLAO"	:	1129854,
    "CUSCO"	    :	1357075,
    "HUANCAVELICA":	365317,
    "HUANUCO"	:	760267,
    "ICA"	    :	975182,
    "JUNIN"	    :	1361467,
    "LA LIBERTAD":	2016771,
    "LAMBAYEQUE":	1310785,
    "LIMA"	    :	10628470,
    "LORETO"	:	1027559,
    "MADRE DE DIOS":173811,
    "MOQUEGUA"	:	192740,
    "PASCO"	    :	271904,
    "PIURA"	    :	2047954,
    "PUNO"	    :	1237997,
    "SAN MARTIN":	899648,
    "TACNA"	    :	370974,
    "TUMBES"	:	251521,
    "UCAYALI"	:	589110,
    "PERU"  	:	32625948,
}

### 6.2. Procesar el dataset

In [13]:
vacfal_dep['no_habitantes'] = vacfal_dep['departamentos'].map(dic_dep)

# Mortality rate per 100k per department
vacfal_dep['tasa_mortalidad'] = round((vacfal_dep['fallecidos']/vacfal_dep['no_habitantes'])*100000,2)

# % of people fully vaccinated per department
vacfal_dep['vac_porcentaje'] = round((vacfal_dep['dosis_2']*100)/vacfal_dep['no_habitantes'],2)

In [14]:
print(f'Total habitantes: {vacfal_dep.no_habitantes.sum()}')
print(f'Total fallecidos: {vacfal_dep.fallecidos.sum()}')
print(f'Total vacunados con 1 dosis: {vacfal_dep.dosis_1.sum()}')
print(f'Total vacunados con 2 dosis: {vacfal_dep.dosis_2.sum()}')
print(f'Total vacunados con 3 dosis: {vacfal_dep.dosis_3.sum()}')
print(f'Porcentaje de la población vacunada: {round(vacfal_dep.dosis_2.sum()*100/vacfal_dep.no_habitantes.sum(),2)}')

Total habitantes: 32625948
Total fallecidos: 202225
Total vacunados con 1 dosis: 23930182
Total vacunados con 2 dosis: 20587547
Total vacunados con 3 dosis: 2235644
Porcentaje de la población vacunada: 63.1


In [15]:
vacfal_dep.dtypes

departamentos       object
fallecidos           int64
dosis_1              int64
dosis_2              int64
dosis_3              int64
no_habitantes        int64
tasa_mortalidad    float64
vac_porcentaje     float64
dtype: object

In [16]:
vacfal_dep

,departamentos,fallecidos,dosis_1,dosis_2,dosis_3,no_habitantes,tasa_mortalidad,vac_porcentaje
0,AMAZONAS,1280,254079,214434,18043,426806,299.90,50.24
1,ANCASH,6751,907529,824080,84031,1180638,571.81,69.80
2,APURIMAC,1526,335102,281123,25093,430736,354.28,65.27
3,AREQUIPA,9747,1140676,1018102,120436,1497438,650.91,67.99
4,AYACUCHO,2157,405434,307663,15034,668213,322.80,46.04
5,CAJAMARCA,4175,1005794,826116,46189,1453711,287.20,56.83
6,CALLAO,10091,1039222,945876,128395,1129854,893.12,83.72
7,CUSCO,4829,952209,799366,61303,1357075,355.84,58.90
8,HUANCAVELICA,1183,238716,182878,12699,365317,323.83,50.06
9,HUANUCO,2715,499979,392333,31353,760267,357.11,51.60


## 7. Guardar el dataset

In [17]:
vacfal_dep.to_csv('Data/DP2_TOTAL_vacunados_y_fallecidos_x_departamento.csv',index = False)